# Introduction

## Objectif du Notebook
L'objectif de ce notebook est de développer et d'évaluer des modèles avancés d'analyse de sentiments en utilisant des architectures de réseaux de neurones convolutionnels (CNN) et de réseaux de neurones récurrents (LSTM). Ces modèles seront entraînés sur un jeu de données de tweets et utiliseront des embeddings textuels pré-entraînés tels que Word2Vec et FastText pour capturer les relations sémantiques entre les mots.

## Présentation des Méthodes CNN et LSTM
Le Convolutional Neural Network (CNN) et le Long Short-Term Memory (LSTM) sont deux types d'architectures de réseaux de neurones particulièrement adaptées au traitement des données séquentielles telles que le texte.

### CNN (Convolutional Neural Network)
Les CNN sont bien connus pour leur capacité à détecter des motifs locaux dans les données séquentielles, ce qui les rend efficaces pour capturer des n-grams de mots dans les textes. Dans ce notebook, nous utiliserons un CNN pour analyser les séquences de mots des tweets et en extraire des caractéristiques pertinentes pour la classification de sentiments.

### LSTM (Long Short-Term Memory)
Les LSTM sont un type de réseau de neurones récurrents (RNN) capable de capturer des dépendances à long terme dans des séquences de données. Cela les rend particulièrement efficaces pour comprendre le contexte complet d'une phrase ou d'un tweet, en tenant compte de l'ordre des mots et des relations temporelles.

## Plan du Notebook
Ce notebook est structuré de manière à suivre un pipeline complet de traitement des données, d'entraînement des modèles et d'évaluation des performances :
1. **Chargement et Préparation des Données** : Nous allons commencer par charger les données, les nettoyer, les tokeniser et les préparer pour les modèles CNN et LSTM.
2. **Préparation des Embeddings Textuels** : Nous utiliserons Word2Vec et FastText pour convertir les séquences de mots en vecteurs denses.
3. **Construction du Modèle CNN** : Nous définirons un modèle CNN pour la tâche de classification de sentiments.
4. **Construction du Modèle LSTM** : Nous définirons un modèle LSTM pour la même tâche.
5. **Entrainement des modèles** : Nous entrainerons les deux modèles.
6. **Comparaison des modèles** : Nous allons comparer les différents modèles.
7. **Export du meilleur modèle** : Nous allons exporter le modèle le plus performant pour notre analyse des sentiments.


# 1. Chargement et Préparation des Données

## 1.1 Importation des bibliothèques nécessaires


In [6]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gensim.downloader as api
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


## 1.2 Chargement des données et transformation de la cible target


In [8]:
# Charger les données
data = pd.read_csv('../data/database_p7_rework.csv')

# Transformation des labels : 0 reste 0 et 4 devient 1
data['target_binary'] = data['target'].apply(lambda x: 0 if x == 0 else 1)

# Vérification des transformations
print("Unique values in target_binary:", data['target_binary'].unique())

Unique values in target_binary: [0 1]


# 2 Tokenization des textes lemmatisés et stemmés

In [9]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Fonction pour préparer les séquences
def prepare_sequences(texts, tokenizer, maxlen=100):
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=maxlen)

# Tokenization
tokenizer_lemma = Tokenizer(num_words=5000)
tokenizer_lemma.fit_on_texts(data['text_lemmatized'])

tokenizer_stem = Tokenizer(num_words=5000)
tokenizer_stem.fit_on_texts(data['text_stemmed'])



# Sélection d'un échantillon équilibré de 16 000 données (8 000 par classe)
sample_data = data.groupby('target_binary', group_keys=False).apply(lambda x: x.sample(8000, random_state=42))

# Fonction pour préparer les séquences avec une longueur maximale de 100
def prepare_sequences(texts, tokenizer, maxlen=100):  # Réduire maxlen à 100
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=maxlen)

# Préparation des séquences avec la longueur max de 100
X_lemma = prepare_sequences(sample_data['text_lemmatized'], tokenizer_lemma, maxlen=100)
X_stem = prepare_sequences(sample_data['text_stemmed'], tokenizer_stem, maxlen=100)

y_sample = sample_data['target_binary'].values

# Affichage de confirmation
print("Données scindées en ensemble d'entraînement et de test pour lemmatisation et stemming.")


Données scindées en ensemble d'entraînement et de test pour lemmatisation et stemming.


# 3 Chargement des modèles pré-entraînés (Word2Vec et FastText)

In [9]:
import gensim.downloader as api

# Charger les modèles Word2Vec et FastText pré-entraînés avec des vecteurs de dimension 300
w2v_model = api.load('word2vec-google-news-300')
ft_model = api.load('fasttext-wiki-news-subwords-300')

print("Modèles FastText et Word2Vec chargés avec succès.")


Modèles FastText et Word2Vec chargés avec succès.


# 5 : Génération des embeddings FastText et Word2Vec

## Embeddings Word2Vec & Fasttext

# 6 : Création des matrices d'embeddings

In [15]:
# Fonction modifiée pour générer des embeddings
def get_embeddings(sequences, tokenizer, model, vector_size=300, max_length=100):
    embeddings = []
    for seq in sequences:
        vectors = []
        for word_idx in seq:  # Utiliser les indices des tokens
            word = tokenizer.index_word.get(word_idx)  # Récupérer le mot correspondant à l'indice
            if word in model:
                vectors.append(model[word])
            else:
                vectors.append(np.zeros(vector_size))  # Ajouter des zéros pour les mots non trouvés
        # S'assurer que la séquence a bien max_length
        if len(vectors) < max_length:
            vectors += [np.zeros(vector_size)] * (max_length - len(vectors))
        elif len(vectors) > max_length:
            vectors = vectors[:max_length]
        embeddings.append(vectors)
    return np.array(embeddings)


# 7 : Préparation des données pour l'entraînement et la validation

In [17]:
# Générer les embeddings avec une longueur de séquence de 100
X_lemma_sample = prepare_sequences(sample_data['text_lemmatized'], tokenizer_lemma, maxlen=100)
X_stem_sample = prepare_sequences(sample_data['text_stemmed'], tokenizer_stem, maxlen=100)

# Générer les embeddings
X_lemma_ft = get_embeddings(X_lemma_sample, tokenizer_lemma, ft_model, vector_size=300)
X_stem_ft = get_embeddings(X_stem_sample, tokenizer_stem, ft_model, vector_size=300)
X_lemma_w2v = get_embeddings(X_lemma_sample, tokenizer_lemma, w2v_model, vector_size=300)
X_stem_w2v = get_embeddings(X_stem_sample, tokenizer_stem, w2v_model, vector_size=300)

# Reformater en (batch_size, max_length, embedding_dim)
X_lemma_w2v = X_lemma_w2v.reshape(X_lemma_w2v.shape[0], 100, 300)
X_stem_w2v = X_stem_w2v.reshape(X_stem_w2v.shape[0], 100, 300)
X_lemma_ft = X_lemma_ft.reshape(X_lemma_ft.shape[0], 100, 300)
X_stem_ft = X_stem_ft.reshape(X_stem_ft.shape[0], 100, 300)
# Cible (target) - identique pour toutes les variantes (FastText/Word2Vec, Lem/Stemming)
y_sample = sample_data['target_binary'].values  # y_sample correspond maintenant à la cible correcte de l'échantillon de 16 000 données

# Split en données d'entraînement et de test avec stratification
X_train_lemma_w2v, X_test_lemma_w2v, y_train_lemma_w2v, y_test_lemma_w2v = train_test_split(X_lemma_w2v, y_sample, test_size=0.2, random_state=42, stratify=y_sample)
X_train_stem_w2v, X_test_stem_w2v, y_train_stem_w2v, y_test_stem_w2v = train_test_split(X_stem_w2v, y_sample, test_size=0.2, random_state=42, stratify=y_sample)
X_train_lemma_ft, X_test_lemma_ft, y_train_lemma_ft, y_test_lemma_ft = train_test_split(X_lemma_ft, y_sample, test_size=0.2, random_state=42, stratify=y_sample)
X_train_stem_ft, X_test_stem_ft, y_train_stem_ft, y_test_stem_ft = train_test_split(X_stem_ft, y_sample, test_size=0.2, random_state=42, stratify=y_sample)

print("Données prêtes pour l'entraînement et le test.")


Données prêtes pour l'entraînement et le test.


# 8 : Définition de l'architecture du modèle CNN

In [39]:
import mlflow
import mlflow.keras
import time
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization, LeakyReLU, PReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import ParameterGrid

def create_cnn_model(max_length=100, num_filters=128, kernel_size=5, dropout_rate=0.2, activation_type='relu'):
    model = Sequential()

    # Couche de convolution et normalisation
    model.add(Conv1D(num_filters, kernel_size=kernel_size, activation='relu', input_shape=(max_length, 300)))
    model.add(BatchNormalization())
    model.add(GlobalMaxPooling1D())
    
    # Ajout de l'activation dynamique dans les couches denses
    if activation_type == 'relu':
        model.add(Dense(128, activation='relu'))
    elif activation_type == 'leaky_relu':
        model.add(Dense(128))
        model.add(LeakyReLU(alpha=0.1))
    elif activation_type == 'prelu':
        model.add(Dense(128))
        model.add(PReLU())

    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model



# Fonction pour entraîner et loguer un modèle CNN avec GridSearch et les activations avancées
def train_and_log_cnn(X_train, y_train, X_test, y_test, experiment_name, param_grid, max_length=100):
    mlflow.set_experiment(experiment_name)
    
    best_model = None
    best_accuracy = 0
    best_params = None

    # Parcourir chaque combinaison d'hyperparamètres
    for params in ParameterGrid(param_grid):
        num_filters = params['num_filters']
        kernel_size = params['kernel_size']
        dropout_rate = params['dropout_rate']
        activation_type = params['activation_type']

        with mlflow.start_run(run_name=f"CNN_filters={num_filters}_kernel={kernel_size}_dropout={dropout_rate}_activation={activation_type}"):

            # Créer le modèle CNN avec les hyperparamètres courants
            model = create_cnn_model(max_length=max_length, num_filters=num_filters, kernel_size=kernel_size, dropout_rate=dropout_rate, activation_type=activation_type)

            # Early stopping pour éviter l'overfitting
            early_stopping = EarlyStopping(monitor='val_loss', patience=4)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

            # Entraîner le modèle
            start_time = time.time()
            history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr], verbose=1)
            training_time = time.time() - start_time

            # Prédictions et évaluation
            y_pred = (model.predict(X_test) > 0.5).astype("int32")
            y_pred_proba = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc_score = roc_auc_score(y_test, y_pred_proba)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)

            # Matrice de confusion
            conf_matrix = confusion_matrix(y_test, y_pred)

            # Loguer les hyperparamètres dans MLFlow
            mlflow.log_param("num_filters", num_filters)
            mlflow.log_param("kernel_size", kernel_size)
            mlflow.log_param("dropout_rate", dropout_rate)
            mlflow.log_param("activation_type", activation_type)

            # Loguer les métriques dans MLFlow
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("auc", auc_score)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("training_time", training_time)

            # Sauvegarder le modèle avec MLFlow
            mlflow.keras.log_model(model, f"cnn_model_{num_filters}_{kernel_size}_{dropout_rate}")

            # Sauvegarder et loguer la matrice de confusion
            plt.figure(figsize=(6, 4))
            sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
            plt.xlabel('Prédictions')
            plt.ylabel('Vérités')
            plt.title(f"Matrice de Confusion - CNN")
            conf_matrix_path = f"./matrice/confusion_matrix_cnn_filters={num_filters}_kernel={kernel_size}_dropout={dropout_rate}.png"
            plt.savefig(conf_matrix_path)
            mlflow.log_artifact(conf_matrix_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook

            # Sauvegarder et loguer la courbe ROC
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            plt.figure()
            plt.plot(fpr, tpr, label=f"ROC curve (AUC = {auc_score:.2f})")
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title("ROC Curve")
            plt.legend(loc="best")
            roc_curve_path = f"./matrice/roc_curve_cnn_filters={num_filters}_kernel={kernel_size}_dropout={dropout_rate}.png"
            plt.savefig(roc_curve_path)
            mlflow.log_artifact(roc_curve_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook

            # Comparer pour garder le meilleur modèle
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = params
                best_model = model

    print(f"Meilleurs paramètres : {best_params} avec une accuracy de {best_accuracy:.4f}")

    # Retourner le meilleur modèle
    return best_model, best_params



# GridSearch pour le modèle CNN avec les hyperparamètres supplémentaires
param_grid_cnn = {
    'num_filters': [64, 128, 256],   # Filtres à tester
    'kernel_size': [3, 5],        # Tailles de kernel à tester
    'dropout_rate': [0.2, 0.5],      # Dropout à tester
    'activation_type': ['relu', 'leaky_relu', 'prelu']  # Activations à tester
}

# 9 : Entrainement des modèles CNN

In [41]:
# Entraîner le modèle avec les données correctement formatées
best_cnn_model_lemma_w2v, best_params_lemma_w2v = train_and_log_cnn(
    X_train_lemma_w2v, y_train_lemma_w2v, X_test_lemma_w2v, y_test_lemma_w2v, 
    experiment_name="CNN_Word2Vec_Lemmatization", param_grid=param_grid_cnn, max_length=100
)


Traceback (most recent call last):
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^^^^^^^^^^^^^^^^^

Epoch 1/20
200/200 [==============================] - 12s 54ms/step - loss: 0.6688 - accuracy: 0.6392 - val_loss: 0.6532 - val_accuracy: 0.7163 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 11s 53ms/step - loss: 0.5128 - accuracy: 0.7484 - val_loss: 0.5739 - val_accuracy: 0.7437 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 11s 53ms/step - loss: 0.4514 - accuracy: 0.7874 - val_loss: 0.5181 - val_accuracy: 0.7478 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 11s 53ms/step - loss: 0.4048 - accuracy: 0.8176 - val_loss: 0.5325 - val_accuracy: 0.7422 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 11s 53ms/step - loss: 0.3535 - accuracy: 0.8430 - val_loss: 0.5510 - val_accuracy: 0.7444 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 11s 53ms/step - loss: 0.2395 - accuracy: 0.9042 - val_loss: 0.6714 - val_accuracy: 0.7372 - lr: 2.0000e-04
Epoch 7/20
100/100 [==============================] - 

2024/10/12 01:02:34 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp7f0ij5cy\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp7f0ij5cy\model\data\model\assets
2024/10/12 01:02:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Epoch 1/20
200/200 [==============================] - 33s 163ms/step - loss: 0.7957 - accuracy: 0.5780 - val_loss: 0.6716 - val_accuracy: 0.6528 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 32s 161ms/step - loss: 0.5797 - accuracy: 0.6948 - val_loss: 0.5940 - val_accuracy: 0.7350 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 32s 161ms/step - loss: 0.5060 - accuracy: 0.7600 - val_loss: 0.5277 - val_accuracy: 0.7469 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 32s 161ms/step - loss: 0.4411 - accuracy: 0.7994 - val_loss: 0.5250 - val_accuracy: 0.7437 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 32s 161ms/step - loss: 0.3734 - accuracy: 0.8377 - val_loss: 0.5307 - val_accuracy: 0.7494 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 32s 161ms/step - loss: 0.3061 - accuracy: 0.8737 - val_loss: 0.6309 - val_accuracy: 0.7441 - lr: 0.0010
Epoch 7/20
200/200 [==============================] 

2024/10/12 02:46:03 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpvkheszfj\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpvkheszfj\model\data\model\assets
2024/10/12 02:46:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'relu', 'dropout_rate': 0.5, 'kernel_size': 3, 'num_filters': 128} avec une accuracy de 0.7541


In [42]:
#Lemma + FastText
best_cnn_model_lemma_ft, best_params_lemma_ft = train_and_log_cnn(
    X_train_lemma_ft, y_train_lemma_ft, X_test_lemma_ft, y_test_lemma_ft, 
    experiment_name="CNN_FastText_Lemmatization", param_grid=param_grid_cnn, max_length=100
)


Traceback (most recent call last):
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^^^^^^^^^^^^^^^^^

Epoch 1/20
200/200 [==============================] - 10s 45ms/step - loss: 0.6721 - accuracy: 0.6129 - val_loss: 0.6671 - val_accuracy: 0.6919 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 9s 43ms/step - loss: 0.5404 - accuracy: 0.7254 - val_loss: 0.5971 - val_accuracy: 0.7381 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 9s 43ms/step - loss: 0.4876 - accuracy: 0.7618 - val_loss: 0.5306 - val_accuracy: 0.7447 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 9s 43ms/step - loss: 0.4463 - accuracy: 0.7922 - val_loss: 0.5058 - val_accuracy: 0.7516 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 9s 43ms/step - loss: 0.4111 - accuracy: 0.8112 - val_loss: 0.5104 - val_accuracy: 0.7563 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 9s 43ms/step - loss: 0.3706 - accuracy: 0.8371 - val_loss: 0.5491 - val_accuracy: 0.7519 - lr: 0.0010
Epoch 7/20
200/200 [==============================] - 9s 43ms/s

2024/10/12 02:47:23 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp60plqloi\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp60plqloi\model\data\model\assets
2024/10/12 02:47:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Epoch 1/20
200/200 [==============================] - 33s 158ms/step - loss: 0.7360 - accuracy: 0.5612 - val_loss: 0.6824 - val_accuracy: 0.6463 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 31s 157ms/step - loss: 0.6262 - accuracy: 0.6413 - val_loss: 0.6558 - val_accuracy: 0.6413 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 31s 157ms/step - loss: 0.5634 - accuracy: 0.7128 - val_loss: 0.5750 - val_accuracy: 0.7466 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 31s 157ms/step - loss: 0.5041 - accuracy: 0.7553 - val_loss: 0.5369 - val_accuracy: 0.7431 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 31s 157ms/step - loss: 0.4598 - accuracy: 0.7859 - val_loss: 0.5151 - val_accuracy: 0.7534 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 31s 157ms/step - loss: 0.4164 - accuracy: 0.8130 - val_loss: 0.5133 - val_accuracy: 0.7584 - lr: 0.0010
Epoch 7/20
200/200 [==============================] 

2024/10/12 04:34:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp14l3tk2t\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp14l3tk2t\model\data\model\assets
2024/10/12 04:34:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'leaky_relu', 'dropout_rate': 0.5, 'kernel_size': 3, 'num_filters': 256} avec une accuracy de 0.7678


In [43]:
# Modèle 3 : Word2Vec + Stemming
best_cnn_model_stem_w2v, best_params_stem_w2v = train_and_log_cnn(
    X_train_stem_w2v, y_train_stem_w2v, X_test_stem_w2v, y_test_stem_w2v, 
    experiment_name="CNN_Word2Vec_Stemming", param_grid=param_grid_cnn, max_length=100
)


Traceback (most recent call last):
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^^^^^^^^^^^^^^^^^

Epoch 1/20
200/200 [==============================] - 10s 43ms/step - loss: 0.6841 - accuracy: 0.6321 - val_loss: 0.6519 - val_accuracy: 0.6628 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 8s 41ms/step - loss: 0.5333 - accuracy: 0.7311 - val_loss: 0.5848 - val_accuracy: 0.6972 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 8s 41ms/step - loss: 0.4828 - accuracy: 0.7640 - val_loss: 0.5370 - val_accuracy: 0.7275 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 8s 41ms/step - loss: 0.4358 - accuracy: 0.7966 - val_loss: 0.5421 - val_accuracy: 0.7275 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 8s 41ms/step - loss: 0.3886 - accuracy: 0.8241 - val_loss: 0.5759 - val_accuracy: 0.7319 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 8s 41ms/step - loss: 0.2814 - accuracy: 0.8825 - val_loss: 0.6163 - val_accuracy: 0.7372 - lr: 2.0000e-04
Epoch 7/20
100/100 [==============================] - 1s 8m

2024/10/12 04:36:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpzr090sj7\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpzr090sj7\model\data\model\assets
2024/10/12 04:36:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Epoch 1/20
200/200 [==============================] - 30s 147ms/step - loss: 0.7827 - accuracy: 0.5666 - val_loss: 0.6853 - val_accuracy: 0.6403 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 29s 145ms/step - loss: 0.6139 - accuracy: 0.6669 - val_loss: 0.6139 - val_accuracy: 0.7337 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 29s 145ms/step - loss: 0.5449 - accuracy: 0.7223 - val_loss: 0.5520 - val_accuracy: 0.7056 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 29s 146ms/step - loss: 0.4898 - accuracy: 0.7644 - val_loss: 0.5224 - val_accuracy: 0.7391 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 29s 146ms/step - loss: 0.4243 - accuracy: 0.8063 - val_loss: 0.5365 - val_accuracy: 0.7378 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 29s 146ms/step - loss: 0.3547 - accuracy: 0.8473 - val_loss: 0.5864 - val_accuracy: 0.7303 - lr: 0.0010
Epoch 7/20
200/200 [==============================] 

2024/10/12 06:11:20 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp61n2anki\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp61n2anki\model\data\model\assets
2024/10/12 06:11:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'relu', 'dropout_rate': 0.5, 'kernel_size': 3, 'num_filters': 64} avec une accuracy de 0.7434


In [44]:
# Stemming + FastText
best_cnn_model_stem_ft, best_params_stem_ft = train_and_log_cnn(
    X_train_stem_ft, y_train_stem_ft, X_test_stem_ft, y_test_stem_ft, 
    experiment_name="CNN_FastText_Stemming", param_grid=param_grid_cnn, max_length=100
)


Traceback (most recent call last):
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^^^^^^^^^^^^^^^^^

Epoch 1/20
200/200 [==============================] - 9s 40ms/step - loss: 0.6794 - accuracy: 0.6051 - val_loss: 0.6663 - val_accuracy: 0.7041 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 7s 37ms/step - loss: 0.5558 - accuracy: 0.7170 - val_loss: 0.6172 - val_accuracy: 0.7216 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 8s 38ms/step - loss: 0.5073 - accuracy: 0.7509 - val_loss: 0.5256 - val_accuracy: 0.7422 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 7s 37ms/step - loss: 0.4672 - accuracy: 0.7798 - val_loss: 0.5255 - val_accuracy: 0.7391 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 8s 38ms/step - loss: 0.4308 - accuracy: 0.8023 - val_loss: 0.5443 - val_accuracy: 0.7347 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 7s 37ms/step - loss: 0.3922 - accuracy: 0.8224 - val_loss: 0.5694 - val_accuracy: 0.7381 - lr: 0.0010
Epoch 7/20
200/200 [==============================] - 8s 38ms/st

2024/10/12 06:12:32 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmprr29g4_n\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmprr29g4_n\model\data\model\assets
2024/10/12 06:12:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Epoch 1/20
200/200 [==============================] - 27s 129ms/step - loss: 0.7596 - accuracy: 0.5539 - val_loss: 0.6892 - val_accuracy: 0.6244 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 26s 130ms/step - loss: 0.6352 - accuracy: 0.6371 - val_loss: 0.6551 - val_accuracy: 0.7116 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 26s 130ms/step - loss: 0.5890 - accuracy: 0.6873 - val_loss: 0.5910 - val_accuracy: 0.7138 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 26s 129ms/step - loss: 0.5306 - accuracy: 0.7366 - val_loss: 0.5507 - val_accuracy: 0.7337 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 26s 130ms/step - loss: 0.4911 - accuracy: 0.7671 - val_loss: 0.5318 - val_accuracy: 0.7312 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 26s 129ms/step - loss: 0.4409 - accuracy: 0.7980 - val_loss: 0.5373 - val_accuracy: 0.7287 - lr: 0.0010
Epoch 7/20
200/200 [==============================] 

2024/10/12 07:46:56 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp0gi3cd1m\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp0gi3cd1m\model\data\model\assets
2024/10/12 07:47:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'prelu', 'dropout_rate': 0.5, 'kernel_size': 3, 'num_filters': 256} avec une accuracy de 0.7528


# 10. Définition de l'architecture du modèle LSTM

In [20]:
import mlflow
import mlflow.keras
import time
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, LeakyReLU, PReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import ParameterGrid

# Fonction pour créer un modèle LSTM sans la couche Bidirectionnelle
def create_lstm_model(input_shape, lstm_units=128, dropout_rate=0.2, activation_type='relu'):
    model = Sequential()

    # Couche LSTM simple
    model.add(LSTM(lstm_units, return_sequences=False, input_shape=input_shape))
    
    # Batch Normalization après la couche LSTM
    model.add(BatchNormalization())

    # Activation alternative
    if activation_type == 'leakyrelu':
        model.add(LeakyReLU())
    elif activation_type == 'prelu':
        model.add(PReLU())
    else:
        model.add(Dense(128, activation=activation_type))

    # Dropout pour éviter le surapprentissage
    model.add(Dropout(dropout_rate))
    
    # Couche Dense
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    
    # Couche de sortie
    model.add(Dense(1, activation='sigmoid'))

    # Compilation du modèle
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Fonction pour entraîner et loguer un modèle LSTM avec GridSearch et les métriques avancées
def train_and_log_lstm(X_train, y_train, X_test, y_test, experiment_name, param_grid, input_shape, max_length=300):
    mlflow.set_experiment(experiment_name)
    
    best_model = None
    best_accuracy = 0
    best_params = None

    # Parcourir chaque combinaison d'hyperparamètres
    for params in ParameterGrid(param_grid):
        lstm_units = params['lstm_units']
        dropout_rate = params['dropout_rate']
        activation_type = params['activation_type']

        with mlflow.start_run(run_name=f"LSTM_units={lstm_units}_dropout={dropout_rate}_activation={activation_type}"):

            # Créer le modèle LSTM avec les hyperparamètres courants
            model = create_lstm_model(input_shape=input_shape, lstm_units=lstm_units, dropout_rate=dropout_rate, activation_type=activation_type)

            # Early stopping pour éviter l'overfitting
            early_stopping = EarlyStopping(monitor='val_loss', patience=4)

            # Réduction du taux d'apprentissage lorsque la validation stagne
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

            # Entraîner le modèle
            start_time = time.time()
            history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr], verbose=1)
            training_time = time.time() - start_time

            # Prédictions et évaluation
            y_pred = (model.predict(X_test) > 0.5).astype("int32")
            y_pred_proba = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc_score = roc_auc_score(y_test, y_pred_proba)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)

            # Matrice de confusion
            conf_matrix = confusion_matrix(y_test, y_pred)

            # Loguer les hyperparamètres dans MLFlow
            mlflow.log_param("lstm_units", lstm_units)
            mlflow.log_param("dropout_rate", dropout_rate)
            mlflow.log_param("activation_type", activation_type)

            # Loguer les métriques dans MLFlow
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("auc", auc_score)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("training_time", training_time)

            # Sauvegarder le modèle avec MLFlow
            mlflow.keras.log_model(model, f"lstm_model_{lstm_units}_{dropout_rate}_{activation_type}")

            # Sauvegarder et loguer la matrice de confusion
            plt.figure(figsize=(6, 4))
            sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
            plt.xlabel('Prédictions')
            plt.ylabel('Vérités')
            plt.title(f"Matrice de Confusion - LSTM")
            conf_matrix_path = f"./matrice/confusion_matrix_lstm_units={lstm_units}_dropout={dropout_rate}_activation={activation_type}.png"
            plt.savefig(conf_matrix_path)
            mlflow.log_artifact(conf_matrix_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook
            
            # Sauvegarder et loguer la courbe ROC
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            plt.figure()
            plt.plot(fpr, tpr, label=f"ROC curve (AUC = {auc_score:.2f})")
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title("ROC Curve")
            plt.legend(loc="best")
            roc_curve_path = f"./matrice/roc_curve_lstm_units={lstm_units}_dropout={dropout_rate}_activation={activation_type}.png"
            plt.savefig(roc_curve_path)
            mlflow.log_artifact(roc_curve_path)
            plt.close()  # Fermer la figure pour éviter l'affichage dans le notebook

            # Comparer pour garder le meilleur modèle
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = params
                best_model = model

    print(f"Meilleurs paramètres : {best_params} avec une accuracy de {best_accuracy:.4f}")

    # Retourner le meilleur modèle
    return best_model, best_params

# GridSearch pour le modèle LSTM avec les hyperparamètres
param_grid_lstm = {
    'lstm_units': [64, 128, 256],      # Nombre de cellules LSTM
    'dropout_rate': [0.2, 0.5],        # Taux de Dropout
    'activation_type': ['relu', 'leakyrelu', 'prelu']  # Type d'activation
}


## 11 : Entraînement des Modèles LSTM 

In [26]:
input_shape = (100, 300)  # max_length=100, embedding_dim=300

In [38]:


# Entraîner avec Word2Vec + Lemmatization
best_lstm_model_lemma_w2v, best_params_lemma_w2v = train_and_log_lstm(
    X_train_lemma_w2v, y_train_lemma_w2v, X_test_lemma_w2v, y_test_lemma_w2v,
    experiment_name="LSTM_Word2Vec_Lemmatization", param_grid=param_grid_lstm, input_shape=input_shape
)

Traceback (most recent call last):
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^^^^^^^^^^^^^^^^^

Epoch 1/20
200/200 [==============================] - 54s 263ms/step - loss: 0.5908 - accuracy: 0.6845 - val_loss: 0.6320 - val_accuracy: 0.7234 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 51s 256ms/step - loss: 0.5154 - accuracy: 0.7503 - val_loss: 0.5424 - val_accuracy: 0.7563 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 51s 255ms/step - loss: 0.4914 - accuracy: 0.7647 - val_loss: 0.5516 - val_accuracy: 0.7053 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 51s 255ms/step - loss: 0.4678 - accuracy: 0.7752 - val_loss: 0.5067 - val_accuracy: 0.7481 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 51s 254ms/step - loss: 0.4440 - accuracy: 0.7894 - val_loss: 0.5207 - val_accuracy: 0.7447 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 51s 254ms/step - loss: 0.4205 - accuracy: 0.8049 - val_loss: 0.6014 - val_accuracy: 0.7384 - lr: 0.0010
Epoch 7/20
200/200 [==============================] 

2024/10/11 14:16:51 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpapd9f4ge\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpapd9f4ge\model\data\model\assets
2024/10/11 14:17:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Epoch 1/20
200/200 [==============================] - 417s 2s/step - loss: 0.6369 - accuracy: 0.6593 - val_loss: 0.6724 - val_accuracy: 0.6606 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 412s 2s/step - loss: 0.5444 - accuracy: 0.7269 - val_loss: 0.6083 - val_accuracy: 0.6375 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 417s 2s/step - loss: 0.5205 - accuracy: 0.7459 - val_loss: 0.5342 - val_accuracy: 0.7497 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 425s 2s/step - loss: 0.5080 - accuracy: 0.7511 - val_loss: 0.5451 - val_accuracy: 0.7456 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 415s 2s/step - loss: 0.4927 - accuracy: 0.7623 - val_loss: 0.5644 - val_accuracy: 0.7262 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 423s 2s/step - loss: 0.4579 - accuracy: 0.7834 - val_loss: 0.4982 - val_accuracy: 0.7491 - lr: 2.0000e-04
Epoch 7/20
200/200 [==============================] - 423s 2

2024/10/12 01:00:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpq_ljr240\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpq_ljr240\model\data\model\assets
2024/10/12 01:01:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'prelu', 'dropout_rate': 0.5, 'lstm_units': 64} avec une accuracy de 0.7656


In [28]:
# Entraîner avec FastText + Lemmatization
best_lstm_model_lemma_ft, best_params_lemma_ft = train_and_log_lstm(
    X_train_lemma_ft, y_train_lemma_ft, X_test_lemma_ft, y_test_lemma_ft,
    experiment_name="LSTM_FastText_Lemmatization", param_grid=param_grid_lstm, input_shape=input_shape
)

Traceback (most recent call last):
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^^^^^^^^^^^^^^^^^

Epoch 1/20
200/200 [==============================] - 11s 44ms/step - loss: 0.5985 - accuracy: 0.6766 - val_loss: 0.6540 - val_accuracy: 0.7244 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 8s 40ms/step - loss: 0.5282 - accuracy: 0.7405 - val_loss: 0.6180 - val_accuracy: 0.6431 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 8s 40ms/step - loss: 0.5011 - accuracy: 0.7566 - val_loss: 0.5805 - val_accuracy: 0.6822 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 8s 40ms/step - loss: 0.4950 - accuracy: 0.7584 - val_loss: 0.5763 - val_accuracy: 0.7003 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 8s 41ms/step - loss: 0.4803 - accuracy: 0.7712 - val_loss: 0.5219 - val_accuracy: 0.7541 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 8s 39ms/step - loss: 0.4653 - accuracy: 0.7776 - val_loss: 0.5197 - val_accuracy: 0.7478 - lr: 0.0010
Epoch 7/20
200/200 [==============================] - 8s 40ms/s

2024/10/10 06:14:20 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpgu0adsco\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpgu0adsco\model\data\model\assets
2024/10/10 06:14:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Epoch 1/20
200/200 [==============================] - 326s 2s/step - loss: 0.6219 - accuracy: 0.6588 - val_loss: 0.6678 - val_accuracy: 0.7184 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 325s 2s/step - loss: 0.5534 - accuracy: 0.7207 - val_loss: 0.6167 - val_accuracy: 0.6944 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 334s 2s/step - loss: 0.5300 - accuracy: 0.7431 - val_loss: 0.5633 - val_accuracy: 0.7209 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 335s 2s/step - loss: 0.5223 - accuracy: 0.7467 - val_loss: 0.5248 - val_accuracy: 0.7416 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 338s 2s/step - loss: 0.5046 - accuracy: 0.7541 - val_loss: 0.6444 - val_accuracy: 0.6916 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 361s 2s/step - loss: 0.4978 - accuracy: 0.7622 - val_loss: 0.6516 - val_accuracy: 0.7084 - lr: 0.0010
Epoch 7/20
200/200 [==============================] - 359s 2s/st

2024/10/10 14:16:32 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpzvccyjk9\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpzvccyjk9\model\data\model\assets
2024/10/10 14:16:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'leakyrelu', 'dropout_rate': 0.2, 'lstm_units': 64} avec une accuracy de 0.7675


In [29]:
# Entraîner avec Word2Vec + Stemming
best_lstm_model_stem_w2v, best_params_stem_w2v = train_and_log_lstm(
    X_train_stem_w2v, y_train_stem_w2v, X_test_stem_w2v, y_test_stem_w2v,
    experiment_name="LSTM_Word2Vec_Stemming", param_grid=param_grid_lstm, input_shape=input_shape
)

Traceback (most recent call last):
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^^^^^^^^^^^^^^^^^

Epoch 1/20
200/200 [==============================] - 48s 231ms/step - loss: 0.6049 - accuracy: 0.6698 - val_loss: 0.6409 - val_accuracy: 0.7262 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 46s 228ms/step - loss: 0.5372 - accuracy: 0.7287 - val_loss: 0.5613 - val_accuracy: 0.7266 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 46s 228ms/step - loss: 0.5084 - accuracy: 0.7487 - val_loss: 0.5218 - val_accuracy: 0.7369 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 45s 228ms/step - loss: 0.4887 - accuracy: 0.7595 - val_loss: 0.5274 - val_accuracy: 0.7375 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 46s 228ms/step - loss: 0.4730 - accuracy: 0.7710 - val_loss: 0.5155 - val_accuracy: 0.7506 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 45s 227ms/step - loss: 0.4447 - accuracy: 0.7890 - val_loss: 0.5253 - val_accuracy: 0.7344 - lr: 0.0010
Epoch 7/20
200/200 [==============================] 

2024/10/10 14:23:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpc3vhuw3m\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpc3vhuw3m\model\data\model\assets
2024/10/10 14:24:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Epoch 1/20
200/200 [==============================] - 368s 2s/step - loss: 0.6277 - accuracy: 0.6546 - val_loss: 0.6601 - val_accuracy: 0.7216 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 374s 2s/step - loss: 0.5633 - accuracy: 0.7109 - val_loss: 0.6031 - val_accuracy: 0.7262 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 375s 2s/step - loss: 0.5416 - accuracy: 0.7289 - val_loss: 0.5844 - val_accuracy: 0.6722 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 374s 2s/step - loss: 0.5187 - accuracy: 0.7377 - val_loss: 0.5134 - val_accuracy: 0.7434 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 376s 2s/step - loss: 0.5052 - accuracy: 0.7532 - val_loss: 0.6013 - val_accuracy: 0.6703 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 373s 2s/step - loss: 0.4911 - accuracy: 0.7602 - val_loss: 0.5359 - val_accuracy: 0.7219 - lr: 0.0010
Epoch 7/20
200/200 [==============================] - 373s 2s/st

2024/10/11 00:54:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpfky01xa5\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpfky01xa5\model\data\model\assets
2024/10/11 00:54:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'relu', 'dropout_rate': 0.5, 'lstm_units': 128} avec une accuracy de 0.7516


In [30]:
# Entraîner avec FastText + Stemming
best_lstm_model_stem_ft, best_params_stem_ft = train_and_log_lstm(
    X_train_stem_ft, y_train_stem_ft, X_test_stem_ft, y_test_stem_ft,
    experiment_name="LSTM_FastText_Stemming", param_grid=param_grid_lstm, input_shape=input_shape
)

Traceback (most recent call last):
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 327, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 421, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1367, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Master_Openclassroom\python\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1360, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^^^^^^^^^^^^^^^^^

Epoch 1/20
200/200 [==============================] - 61s 295ms/step - loss: 0.6030 - accuracy: 0.6716 - val_loss: 0.6600 - val_accuracy: 0.5981 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 58s 289ms/step - loss: 0.5400 - accuracy: 0.7295 - val_loss: 0.5958 - val_accuracy: 0.7412 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 58s 289ms/step - loss: 0.5212 - accuracy: 0.7431 - val_loss: 0.5290 - val_accuracy: 0.7397 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 58s 290ms/step - loss: 0.5068 - accuracy: 0.7502 - val_loss: 0.5204 - val_accuracy: 0.7425 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 58s 290ms/step - loss: 0.4890 - accuracy: 0.7645 - val_loss: 0.5177 - val_accuracy: 0.7372 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 58s 290ms/step - loss: 0.4777 - accuracy: 0.7736 - val_loss: 0.5286 - val_accuracy: 0.7353 - lr: 0.0010
Epoch 7/20
200/200 [==============================] 

2024/10/11 01:03:32 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpo_61fgui\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmpo_61fgui\model\data\model\assets
2024/10/11 01:03:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Epoch 1/20
200/200 [==============================] - 384s 2s/step - loss: 0.6384 - accuracy: 0.6448 - val_loss: 0.6744 - val_accuracy: 0.5609 - lr: 0.0010
Epoch 2/20
200/200 [==============================] - 388s 2s/step - loss: 0.5627 - accuracy: 0.7127 - val_loss: 0.6315 - val_accuracy: 0.6903 - lr: 0.0010
Epoch 3/20
200/200 [==============================] - 390s 2s/step - loss: 0.5430 - accuracy: 0.7281 - val_loss: 0.5864 - val_accuracy: 0.6809 - lr: 0.0010
Epoch 4/20
200/200 [==============================] - 391s 2s/step - loss: 0.5285 - accuracy: 0.7395 - val_loss: 0.5550 - val_accuracy: 0.7194 - lr: 0.0010
Epoch 5/20
200/200 [==============================] - 392s 2s/step - loss: 0.5234 - accuracy: 0.7405 - val_loss: 0.9759 - val_accuracy: 0.5066 - lr: 0.0010
Epoch 6/20
200/200 [==============================] - 390s 2s/step - loss: 0.5108 - accuracy: 0.7477 - val_loss: 0.5773 - val_accuracy: 0.6744 - lr: 0.0010
Epoch 7/20
200/200 [==============================] - 390s 2s/st

2024/10/11 14:08:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp00qs73us\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\trist\AppData\Local\Temp\tmp00qs73us\model\data\model\assets
2024/10/11 14:08:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Meilleurs paramètres : {'activation_type': 'prelu', 'dropout_rate': 0.5, 'lstm_units': 64} avec une accuracy de 0.7559


## Analyse des meilleurs modele


In [76]:
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd

# Créer un client MLflow
client = MlflowClient()

# Liste des run_ids à comparer
run_ids = [
    "0e149cb1aade4ba789ba469cd758e2e4", "d7c0384dc122441d8b021888089e36ad", 
    "323a3367a5f34a5e9dbbc380deb09e0c", "217a0e9918764c46bfffa00ea342fe09",
    "c6ae9082007f40a6b0cf4cb2c1c58dc7", "61711770d0e947ae9806c023633257ca",
    "6bcbdd5919fb406d93ebe4eee5ee93a1", "d15f0469c1f04f608ee597e15276bf29"
]

# Récupérer les métriques des runs et les stocker dans une liste
run_data = []
for run_id in run_ids:
    run = client.get_run(run_id)
    
    # Récupérer l'ID de l'expérimentation associée
    experiment_id = run.info.experiment_id
    experiment = client.get_experiment(experiment_id)
    experiment_name = experiment.name
    
    # Récupérer les métriques du run
    metrics = run.data.metrics
    metrics['run_id'] = run_id  # Ajouter le run_id pour identifier chaque run
    metrics['experiment_name'] = experiment_name  # Ajouter le nom de l'expérimentation
    
    run_data.append(metrics)

# Convertir la liste en DataFrame pour faciliter l'analyse et la comparaison
df = pd.DataFrame(run_data)

# Afficher les résultats
df


,accuracy,auc,f1_score,precision,recall,training_time,run_id,experiment_name
0,0.754062,0.834188,0.753214,0.755821,0.750625,140.503906,0e149cb1aade4ba789ba469cd758e2e4,CNN_Word2Vec_Lemmatization
1,0.752812,0.829275,0.737645,0.785866,0.695000,161.961351,d7c0384dc122441d8b021888089e36ad,CNN_FastText_Stemming
2,0.720000,0.792584,0.714831,0.728275,0.701875,123.168461,323a3367a5f34a5e9dbbc380deb09e0c,CNN_Word2Vec_Stemming
3,0.767813,0.839627,0.769039,0.764997,0.773125,207.850207,217a0e9918764c46bfffa00ea342fe09,CNN_FastText_Lemmatization
4,0.765625,0.843100,0.760536,0.777415,0.744375,877.881975,c6ae9082007f40a6b0cf4cb2c1c58dc7,LSTM_Word2Vec_Lemmatization
5,0.755938,0.833194,0.751036,0.766428,0.736250,1013.713845,61711770d0e947ae9806c023633257ca,LSTM_FastText_Stemming
6,0.751563,0.831365,0.737364,0.782060,0.697500,1946.616305,6bcbdd5919fb406d93ebe4eee5ee93a1,LSTM_Word2Vec_Stemming
7,0.748437,0.836294,0.719219,0.813733,0.644375,662.600496,d15f0469c1f04f608ee597e15276bf29,LSTM_FastText_Lemmatization


In [78]:
# Attribuer des rangs (de 1 pour le meilleur à 8 pour le moins bon)
df['rank_accuracy'] = df['accuracy'].rank(ascending=False).astype(int)
df['rank_auc'] = df['auc'].rank(ascending=False).astype(int)
df['rank_f1'] = df['f1_score'].rank(ascending=False).astype(int)
df['rank_recall'] = df['recall'].rank(ascending=False).astype(int)
df['rank_training_time'] = df['training_time'].rank(ascending=True).astype(int)  # Le plus bas est le meilleur

# Calculer la somme des rangs pour chaque modèle
df['total_rank'] = df['rank_accuracy'] + df['rank_auc'] + df['rank_f1'] + df['rank_recall'] + df['rank_training_time']

# Trier les modèles par la somme des rangs (plus petite somme = meilleur modèle)
df_sorted = df.sort_values(by='total_rank')

# Afficher le meilleur modèle
best_model = df_sorted.iloc[0]

print("Le meilleur modèle est :")
print(best_model)

Le meilleur modèle est :
accuracy                                      0.767813
auc                                           0.839627
f1_score                                      0.769039
precision                                     0.764997
recall                                        0.773125
training_time                               207.850207
run_id                217a0e9918764c46bfffa00ea342fe09
experiment_name             CNN_FastText_Lemmatization
rank_accuracy                                        1
rank_auc                                             2
rank_f1                                              1
rank_recall                                          1
rank_training_time                                   4
total_rank                                           9
Name: 3, dtype: object


# Entrainement sur tout le dataset du meilleur model plus sauvegarde avec tensorlite

In [13]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Fonction pour préparer les séquences
def prepare_sequences(texts, tokenizer, maxlen=100):
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=maxlen)

# Tokenization
tokenizer_lemma = Tokenizer(num_words=5000)
tokenizer_lemma.fit_on_texts(data['text_lemmatized'])


# Fonction pour préparer les séquences avec une longueur maximale de 100
def prepare_sequences(texts, tokenizer, maxlen=100):  # Réduire maxlen à 100
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=maxlen)

# Préparation des séquences avec la longueur max de 100
X_lemma = prepare_sequences(data['text_lemmatized'], tokenizer_lemma, maxlen=100)

y_sample = data['target_binary'].values

# Affichage de confirmation
print("Preparation donnée faite.")


Preparation donnée faite.


In [16]:
# Diviser les séquences en ensembles d'entraînement et de test
X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(X_lemma, y_sample, test_size=0.2, random_state=42, stratify=y_sample)

In [78]:
import numpy as np

# Générateur d'embeddings pour Keras qui retourne à la fois les séquences et les cibles
def embedding_generator(sequences, labels, tokenizer, model, vector_size=300, max_length=100, batch_size=32):
    while True:
        for i in range(0, len(sequences), batch_size):
            batch_sequences = sequences[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]  # Obtenir les labels correspondants
            batch_embeddings = []
            for seq in batch_sequences:
                vectors = []
                for word_idx in seq:
                    word = tokenizer.index_word.get(word_idx)
                    if word in model:
                        vectors.append(model[word])
                    else:
                        vectors.append(np.zeros(vector_size))  # Ajouter des zéros pour les mots non trouvés
                # S'assurer que la séquence a bien max_length
                if len(vectors) < max_length:
                    vectors += [np.zeros(vector_size)] * (max_length - len(vectors))
                elif len(vectors) > max_length:
                    vectors = vectors[:max_length]
                batch_embeddings.append(vectors)
            yield np.array(batch_embeddings), np.array(batch_labels)  # Retourner les embeddings et les labels

# Préparer les séquences
X_lemma = prepare_sequences(data['text_lemmatized'], tokenizer_lemma, maxlen=100)

# Diviser les séquences en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split
X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(X_lemma, y_sample, test_size=0.2, random_state=42, stratify=y_sample)

# Créer les générateurs avec les cibles
train_generator = embedding_generator(X_train_lemma, y_train_lemma, tokenizer_lemma, ft_model, vector_size=300, max_length=100, batch_size=32)
test_generator = embedding_generator(X_test_lemma, y_test_lemma, tokenizer_lemma, ft_model, vector_size=300, max_length=100, batch_size=32)

print("Générateurs d'embeddings créés.")


Générateurs d'embeddings créés.


In [89]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization, LeakyReLU

# Fonction pour créer un modèle CNN avec les hyperparamètres prédéfinis
def create_cnn_model(max_length=100, embedding_dim=300, num_filters=256, kernel_size=3, dropout_rate=0.5):
    model = Sequential()

    # Couche de convolution et normalisation
    model.add(Conv1D(num_filters, kernel_size=kernel_size, activation='relu', input_shape=(max_length, embedding_dim)))
    model.add(BatchNormalization())
    model.add(GlobalMaxPooling1D())

    # Couche dense avec l'activation Leaky ReLU
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.1))  # Utilisation directe de Leaky ReLU

    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))  # Utilisation de ReLU pour cette couche dense
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))  # Sortie pour un problème de classification binaire
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

print("Modèle de machine learning créé.")


Modèle de machine learning créé.


In [91]:
# Fonction d'entraînement du modèle sans sauvegarde
def train_model_without_saving(model, train_generator, test_generator, X_train_lemma, X_test_lemma):
    # Ajouter les callbacks pour éviter l'overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

    # Entraîner le modèle avec les générateurs
    history = model.fit(
        train_generator, 
        steps_per_epoch=len(X_train_lemma) // 32, 
        validation_data=test_generator, 
        validation_steps=len(X_test_lemma) // 32, 
        epochs=50,  # Limite à 50 époques
        callbacks=[early_stopping, reduce_lr]  # Utiliser les callbacks pour ajuster
    )

    # Stocker les résultats d'entraînement dans des variables
    training_metrics = {
        'accuracy': max(history.history['accuracy']),
        'val_accuracy': max(history.history['val_accuracy']),
        'loss': min(history.history['loss']),
        'val_loss': min(history.history['val_loss'])
    }

    print("Entraînement terminé. Les résultats sont stockés.")
    return training_metrics


In [93]:
import mlflow
import mlflow.keras
import tensorflow as tf

# Fonction pour sauvegarder le modèle et les métriques via MLflow sans gérer manuellement le dossier
def save_model_and_metrics(model, training_metrics):
    # Commencer un run MLflow pour loguer les métriques et hyperparamètres
    with mlflow.start_run():
        # Loguer les hyperparamètres dans MLflow
        mlflow.log_param("num_filters", 256)
        mlflow.log_param("kernel_size", 3)
        mlflow.log_param("dropout_rate", 0.5)
        mlflow.log_param("activation_type", 'leaky_relu')
        mlflow.log_param("max_length", 100)
        mlflow.log_param("embedding_dim", 300)

        # Loguer les métriques d'entraînement et de validation dans MLflow
        mlflow.log_metric("accuracy", training_metrics['accuracy'])
        mlflow.log_metric("val_accuracy", training_metrics['val_accuracy'])
        mlflow.log_metric("loss", training_metrics['loss'])
        mlflow.log_metric("val_loss", training_metrics['val_loss'])

        # Sauvegarder le modèle dans MLflow (MLflow gère automatiquement le dossier)
        model_name = "cnn_model_256_3_0.5"
        mlflow.keras.log_model(model, artifact_path=model_name)

        # Convertir le modèle en TensorFlow Lite
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()

        # Sauvegarder le modèle TensorFlow Lite (dans le répertoire temporaire MLflow ou un chemin par défaut)
        tflite_filename = f"{model_name}.tflite"
        with open(tflite_filename, "wb") as f:
            f.write(tflite_model)

        print(f"Le modèle a été converti et sauvegardé sous {tflite_filename}.")


In [ ]:
# Entraîner le modèle puis sauvegarder après les calculs
model = create_cnn_model(max_length=100, embedding_dim=300, num_filters=256, kernel_size=3, dropout_rate=0.5)

# Entraînement sans sauvegarde immédiate
training_metrics = train_model_without_saving(model, train_generator, test_generator, X_train_lemma, X_test_lemma)


Epoch 1/50
39957/39957 [==============================] - 814s 20ms/step - loss: 0.4698 - accuracy: 0.7824 - val_loss: 0.4278 - val_accuracy: 0.8018 - lr: 0.0010
Epoch 2/50
39957/39957 [==============================] - 793s 20ms/step - loss: 0.4336 - accuracy: 0.8038 - val_loss: 0.4160 - val_accuracy: 0.8083 - lr: 0.0010
Epoch 3/50
39957/39957 [==============================] - 811s 20ms/step - loss: 0.4205 - accuracy: 0.8103 - val_loss: 0.4128 - val_accuracy: 0.8109 - lr: 0.0010
Epoch 4/50
39957/39957 [==============================] - 805s 20ms/step - loss: 0.4125 - accuracy: 0.8147 - val_loss: 0.4110 - val_accuracy: 0.8123 - lr: 0.0010
Epoch 5/50
39957/39957 [==============================] - 817s 20ms/step - loss: 0.4064 - accuracy: 0.8175 - val_loss: 0.4103 - val_accuracy: 0.8136 - lr: 0.0010
Epoch 6/50
39957/39957 [==============================] - 803s 20ms/step - loss: 0.4012 - accuracy: 0.8200 - val_loss: 0.4145 - val_accuracy: 0.8126 - lr: 0.0010
Epoch 7/50
39957/39957 [====

In [ ]:
# Sauvegarder le modèle et les résultats après l'entraînement
save_model_and_metrics(model, training_metrics)